# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import pprint

# Import API key
from api_keys import g_key

%load_ext autoreload
%autoreload 2

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# create a reference to the csv file
cities_csv = "../output_data/cities.csv"
# read the csv file into a dataframe
cities_df = pd.read_csv(cities_csv)
cities_df = cities_df.dropna()
del cities_df["Unnamed: 0"]
cities_df.head()
cities_df.count()

City          559
Lat           559
Lng           559
Max Temp      559
Humidity      559
Cloudiness    559
Wind Speed    559
Country       559
Date          559
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' as locations
locations = cities_df[["Lat", "Lng"]].astype(float)
humidity = cities_df["Humidity"].astype(float)



In [4]:
# Add heatmap layer to map
fig = gmaps.figure(center=(50.0, -35.0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius = 5)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the DataFrame to find your ideal weather condition.
# Drop any rows that don't contain all three conditions

# A max temperature lower than 80 degrees but higher than 70.
city_low_temp = cities_df[cities_df["Max Temp"] > 70] 
city_max_temp = city_low_temp[city_low_temp["Max Temp"] < 80]
# Zero cloudiness
city_cloudless = city_max_temp[city_max_temp["Cloudiness"] == 0]
# Wind speed less than 10 mph
city_wind = city_cloudless[city_cloudless["Wind Speed"] < 40]
# Store into hotel variable name
hotel_df = city_wind
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
93,Kysyl-Syr,63.8986,122.7617,76.91,28,0,4.81,RU,1628224193
251,Jadu,31.9530,12.0261,78.51,23,0,7.18,LY,1628224238
260,Palmas,-10.2128,-48.3603,75.07,53,0,4.61,BR,1628224142
473,Geraldton,-28.7667,114.6000,71.13,49,0,19.57,AU,1628223840
521,Belyy Yar,53.6039,91.3903,76.89,53,0,2.24,RU,1628224301


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
reduced_cities_df = cities_df.loc[(cities_df["Wind Speed"] < 10) & (cities_df["Cloudiness"] == 0) & \
                                   (cities_df["Max Temp"] > 70) & (cities_df["Max Temp"] < 80)].dropna()
reduced_cities_df.count()

City          4
Lat           4
Lng           4
Max Temp      4
Humidity      4
Cloudiness    4
Wind Speed    4
Country       4
Date          4
dtype: int64

In [7]:
hotel_df = city_wind.loc[:,["City", "Country", "Lat", "Lng"]]
hotel_df

,City,Country,Lat,Lng
93,Kysyl-Syr,RU,63.8986,122.7617
251,Jadu,LY,31.9530,12.0261
260,Palmas,BR,-10.2128,-48.3603
473,Geraldton,AU,-28.7667,114.6000
521,Belyy Yar,RU,53.6039,91.3903


In [8]:
import json
import pprint
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    pprint.pprint(response)
    # extract results
    results = response['results']
    print(json.dumps(response, indent=4, sort_keys=True))
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("No hotel")
    print("------------")
    

Retrieving Results for Index 93: Kysyl-Syr.
{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 63.8968847, 'lng': 122.7602181},
                           'viewport': {'northeast': {'lat': 63.89818042989273,
                                                      'lng': 122.7616762298927},
                                        'southwest': {'lat': 63.89548077010728,
                                                      'lng': 122.7589765701073}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/bank-71.png',
              'icon_background_color': '#909CE1',
              'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/bank-intl_pinlet',
              'name': 'Sberbank',
              'opening_hours': {'open_now': False},
              'place_id': 'ChIJQ1P22Q3xiVsRX6aZggRfDMo',
              'plus_code': {'compound_code': 'VQW6+Q3 Kysyl-Syr, Sakha '
      

{'html_attributions': [],
 'next_page_token': 'Aap_uEAcQnPYHxWQpvHjB4ugZaFAbwD4ownVYVnprC2GKBSuR0KtuG5vJY4_Q4u5vIE7jJmcGVQmEKtyehoCLbDas4yOfvb2Zch4q9MCuosb5XLV-yB4iNxwCS15f-9be-VUwy4HWZIcKIdX3CJUCJjGJjVh4MNbTRSzfwBgyFOtkfwY9NYOrhFgsSN6OUAmbV88Coiub9LTSLTxgxjX5-quTHI4P5mlSs1ynl7-9hWYBgvuH1HAkoqn_Pi10D-9ObpnlOv1C1yTzD4ana0-2Onb8x6TOGCIdb4rby8u4sBEHg_ECffH79US5D7nYBrPaDI9uHh5jB9b9GUzkvRIXmBnU74lx2l8SwruPHTWRF_aSWBsNheaoZSx4OP_InpVep-ajV4-w4Z-SGMP8Px9rhqiuJJxuN42V0DdwuShd_Bkc6tC14mgpctKt6Euw3WN6N4',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -28.7738192, 'lng': 114.6082717},
                           'viewport': {'northeast': {'lat': -28.77237427010728,
                                                      'lng': 114.6095486298927},
                                        'southwest': {'lat': -28.77507392989272,
                                                      'lng': 114.6068489701073}}},
              'icon': 'https://maps.gstatic.

{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
No hotel
------------


In [9]:
# Add "Hotel Name" column to DF
hotel_df["Hotel Name"] = "" 
hotel_name = []

# Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   
    
# Set parameters
target_radius = 5000
target_type = "hotel"
params = {
    "key": g_key,
    "radius": target_radius,
    "type": target_type
    }

    # Loop through the hotel_df and get the Lat/Lng for each city
for index, row in hotel_df.iterrows():
    # get coordinates
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    params["location"] = f"{lat},{lng}"
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    # print response
    results = response['results']
    pprint.pprint(response)
    
    # save the hotel name to dataFrame
    hotel_df.loc[index, "Hotel Name"] = results[0]['name']
   
    
    
# Plot the hotels on top of the humidity heatmap with each pin containing the **Hotel Name**, **City**, and **Country**



Retrieving Results for Index 93: Kysyl-Syr.
{'html_attributions': [],
 'results': [{'geometry': {'location': {'lat': 63.89784700000001,
                                        'lng': 122.762485},
                           'viewport': {'northeast': {'lat': 63.90275578379044,
                                                      'lng': 122.7784924237714},
                                        'southwest': {'lat': 63.89293735767683,
                                                      'lng': 122.7464775762286}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
              'icon_background_color': '#7B9EB0',
              'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_pinlet',
              'name': 'Kysyl-Syr',
              'place_id': 'ChIJz2AMkiXwiVsRxFF9ljod_Hw',
              'reference': 'ChIJz2AMkiXwiVsRxFF9ljod_Hw',
              'scope': 'GOOGLE',
              'types': ['locality

{'html_attributions': [],
 'next_page_token': 'Aap_uEC6IuReCtKIbqndsDwv7TqtD6JTb2f6YtxS2RaqDC7kGJUffojuGQGps_C9rQTE5nnxzfzjnSH-gk4OhbpA2o7QlDlaTQqgpFENW9eNd1RmqAj4Gg1E-5OE4ixteaU0LAeI-6ijNIoi6fNQmKYU9Pd_L_dDzSkcg5oHmyvD8efpjdnZ6khwrInrKzsMEFOru73ivxO5OBCfZbOd5if8V19PcEK7DriDfvVSbKbkCnwqlzIxBvGeKpYIHsctLe4c55-crjKfkBpm4caQDUshNd78UdQGaqRNrU9E5K-tHv3ml0cgjqVJsbMTCM8Qkj47AIzcpiKIZH9BVPFcWmKVj1YYwtYHTRjiSQG0GJuJ1EGdCHx-6RO-y3a3GnXoaB2AqKMC89qpXvIT_1A8o5jXS0Uj2_OMdhmDMPCbIkrJgeH3V0PfcUgbDw',
 'results': [{'geometry': {'location': {'lat': 31.9584113, 'lng': 12.0204247},
                           'viewport': {'northeast': {'lat': 31.97080393533256,
                                                      'lng': 12.03861237611902},
                                        'southwest': {'lat': 31.93766866978768,
                                                      'lng': 12.00942995804585}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
     

{'html_attributions': [],
 'next_page_token': 'Aap_uEB5e0aeF4PAn5707tzhniaOWn0p1-I2j0TZVsPGkfIWyTMandWTQNIE2bv_GpVH527G0EyJAtJIsHNmackbcrhUrmuzT-hw1twg3uMn-CUNZAgVqc21KnNlpzUSJJJ0n43VqHhyC46-EBihIrYbuSOl2YvCi-vinbAf4TgGBUjyXDXZjLQPSFGrLotQIH-u2-qZ7W6Z8_dj-7Mng5bDtGzbonXGoe0VStpC3AIxxHG8W1dCQXBk6S4woKiIzzNtkrR9pmWWlHjXqfoZVYCDTEYm45INEIkkAL51lkBI2D6OXr4r5kXzKTNITZXo-hyJKY81pNueCITL925_erMqC6nhTw8vjcHN9AAMn_AVChy4eGd5gNhvtKt1iDiI0ODQuvs_pQs7HzvCfgBilWXBCIyDhYydc1Tj2Y4u9rxgYMARgQKqd27vqg',
 'results': [{'geometry': {'location': {'lat': -10.249091,
                                        'lng': -48.32428580000001},
                           'viewport': {'northeast': {'lat': -10.14902413213807,
                                                      'lng': -48.2749575774645},
                                        'southwest': {'lat': -10.37109528679374,
                                                      'lng': -48.41951107186923}}},
              'icon': 'https://maps.gstatic.com/mapfil

{'html_attributions': [],
 'next_page_token': 'Aap_uEDLTCTS31FAHhRA2Rm0wumjJfdeYfsSjx5nYFaHMyw1jfzqKVUFqFdiTITYPs4PgBm94dZoMuJ_QZCR3Q8dJ9XVXy198qdgK88xhyMwAhHULe8rBdOpxxwZluTNtTu13wTUOsRzLksK8nO7uzWgHINdR47XqSn1955lN-QGpAOovG10Nob_Q3CZ035gIKJVDG_gY9NjmDENPab0rxRLozOokocEM8YXRaDYIMTxjh5phTVjyUMOOdgEz7swRJH--1c98I6GPoo5yztn6tK7rO6ECxmi2GmdbwoClr9vAuxCAZbrfhpSQDD1-HLta_YTKoM-6xFdny6hxF3DKmYgir2IeLYgBRfDqinBIzdTdj74m7jLsGm_HU63Ez4tFa2iWVVa8mym4p8C9M1UtQHmr29LnJzPNwpR5qEOqT9e6ug-ELZ6oM09Xw',
 'results': [{'geometry': {'location': {'lat': -28.7750917, 'lng': 114.609858},
                           'viewport': {'northeast': {'lat': -28.76276761545247,
                                                      'lng': 114.6261532804849},
                                        'southwest': {'lat': -28.78861926424706,
                                                      'lng': 114.5962706435342}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
  

{'html_attributions': [],
 'next_page_token': 'Aap_uECrkLjMXfBRTDa3gh6bOKpN3Dy1YNo8qdN8zMNTS1ezJSif3cdMJUwUQixUoWCyYruXF36x0mP8r6QR6PUHx_qM4ktgi9koBTbuPlXOIBErAUC86-TfxPrZs4NCKbWokSG6SMHVo1fQUKp78cm0C38yYc0H29anpBqlOi7KjvVc1mWvlsPNdJiXRnAMj2_PydznLdK_AWBEhd1jIc3YafX9SiGLl3S-LXFwkTi42CnySmzZQsK-jiTfUNfLOBBrlrnzEgmfpViA5_7gAgRigXcTFoSLtKTDMMVnC20-0XW9289GDay0DBi8YOw2LWjf1FEVVOMTa2-50WD2EnP-m_mM8FqW7NzvBaxrbmIMAexorurbaPMHovcGGNsXktZPUlDtxyfZTGeEJgHyRMqumt8azdTbOXoX5f-24_gZIXW4oecRobXGUw',
 'results': [{'geometry': {'location': {'lat': 53.602625,
                                        'lng': 91.38034789999999},
                           'viewport': {'northeast': {'lat': 53.61961465915024,
                                                      'lng': 91.41240414932443},
                                        'southwest': {'lat': 53.57831778264526,
                                                      'lng': 91.34420495600952}}},
              'icon': 'https://maps.gstatic.com/mapfiles/pl

In [10]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
93,Kysyl-Syr,RU,63.8986,122.7617,Kysyl-Syr
251,Jadu,LY,31.9530,12.0261,Jadu
260,Palmas,BR,-10.2128,-48.3603,Palmas
473,Geraldton,AU,-28.7667,114.6000,Geraldton
521,Belyy Yar,RU,53.6039,91.3903,Belyy Yar


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))